# No-skill tiles

Some classifiers may always return the same value. This can occur when dealing with largely unbiased datasets for instance.

In this notebook, we explore the specific case in which the data distribution and the model predictions follow distributions that are iid.

Considering:
- $\hat{P}(+)$: the probability of a model to yield a positive.
- $\pi_+$: the probability to encounter a data instance corresponding to the positive class.

One can see that ...

In [1]:
%matplotlib tk

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

from sorbetto.performance.two_class_classification_performance import TwoClassClassificationPerformance
from sorbetto.parameterization.parameterization_default import ParameterizationDefault
from sorbetto.flavor.value_flavor import ValueFlavor
from sorbetto.flavor.ranking_flavor import RankingFlavor
from sorbetto.tile.value_tile import ValueTile
from sorbetto.tile.ranking_tile import RankingTile
from sorbetto.core.entity import Entity

In [2]:
def get_perf_no_skill(prior_pos = 0.5, proba_model_pos = 0.5):

    prior_neg = 1 - prior_pos
    proba_model_neg = 1 - proba_model_pos

    ptp = proba_model_pos * prior_pos
    ptn = proba_model_neg * prior_neg
    pfn = proba_model_neg * prior_pos
    pfp = proba_model_pos * prior_neg


    return TwoClassClassificationPerformance(ptp, pfn, pfp, ptn, name = 'No-skill Performance')

perf = get_perf_no_skill(prior_pos = 0.3, proba_model_pos = .3)
print(perf)

TwoClassClassificationPerformance(name=No-skill Performance, ptn=0.09, pfp=0.21, pfn=0.21, ptp=0.48999999999999994)


In [3]:
# Creating an example

param = ParameterizationDefault()
flavor = ValueFlavor(perf)
tile = ValueTile(flavor = flavor, parameterization = param, name = "value tile")

a, b = np.linspace(0,1,100), np.linspace(0,1,100)
a2, b2 = np.meshgrid(a, b)
mat = tile(a2, b2)

fig, ax = plt.subplots()

def display_tile(fig, ax, mat):

    ax.imshow(mat, extent=[0, 1, 0, 1], origin='lower', aspect='auto')
    cs = ax.contour(
        a, b, mat,
        levels=np.linspace(0, 1, 21),
        colors="cornflowerblue"
    )
    ax.clabel(cs, inline=True, fontsize=8)

    # Axis labels
    ax.set_xlabel('a')
    ax.set_ylabel('b')

    ax.set_aspect('equal', adjustable='box')

display_tile(fig, ax, mat)

plt.show()

To explore how the value tile change when the priors and the model probabilities change, the next cell provides you with an interactive plot.

Please note that when $\pi_+ = P(+) = 0.5$, the tile is a uniform square.

You can also check that when $\pi = 0.1$ and $P(+) = 0.9$, you obtain horizontal lines. 

When $\pi = P(+) = 0.9$, you obtain vertical lines.

In [4]:

fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)  # leave space for sliders

a = np.linspace(0, 1, 100)
b = np.linspace(0, 1, 100)
a2, b2 = np.meshgrid(a, b)

prior_ax = fig.add_axes([0.2, 0.1, 0.6, 0.03])
prior_pos_slider = Slider(
    ax=prior_ax,
    label=r'$\pi_+$',
    valmin=0.0,
    valmax=1.0,
    valinit=0.5,
    valstep=0.01,
    orientation='horizontal'
)

proba_ax = fig.add_axes([0.2, 0.05, 0.6, 0.03])
proba_model_slider = Slider(
    ax=proba_ax,
    label='P(+)',
    valmin=0.0,
    valmax=1.0,
    valinit=0.5,
    valstep=0.01,
    orientation='horizontal'
)

def plot_tile(_=None):
    prior_val = prior_pos_slider.val
    proba_model_val = proba_model_slider.val

    perf = get_perf_no_skill(prior_pos=prior_val, proba_model_pos=proba_model_val)
    flavor = ValueFlavor(perf)
    value_tile = ValueTile(flavor=flavor, parameterization=param, name="value tile")
    mat = value_tile(a2, b2)

    ax.clear()
    display_tile(fig, ax, mat)
    fig.canvas.draw_idle()

prior_pos_slider.on_changed(plot_tile)
proba_model_slider.on_changed(plot_tile)

# initial draw
plot_tile()
plt.show()


It is also possible to observe how the tile ranks different no-skill predictors.

In [5]:
entities_list = []
id_dict = []
priors_list = np.linspace(0, 1, 50)
model_probs_list = np.linspace(0, 1, 50)


def encode_id(prior, model_prob):
    return f"{prior:.2f}_{model_prob:.2f}"

i = 0
for prior in priors_list:
    for model_prob in model_probs_list:
        perf = get_perf_no_skill(prior_pos=prior, proba_model_pos=model_prob)
        ent = Entity(perf, name=f"Prior: {prior:.2f}, P(+): {model_prob:.2f}")
        entities_list.append(ent)
        id_dict.append( (encode_id(prior, model_prob), i) )
        i += 1

id_dict = dict(id_dict)

In [6]:
id_entity = 200

flavor = RankingFlavor(entity = entities_list[id_entity],
                       entity_list=entities_list,
                       name = "Rank Flavor - No Skill")

tile = RankingTile(name = "Rank Tile - No Skill", 
                   parameterization=param,
                   flavor=flavor)

In [7]:
fig, ax = plt.subplots()

a = np.linspace(0, 1, 100)
b = np.linspace(0, 1, 100)
a2, b2 = np.meshgrid(a, b)

mat = tile(a2,b2) / len(entities_list)
ax.imshow(mat)

#levels = np.unique(mat).astype(float) - 0.5  # boundaries between integer plateaus

# Axis labels
ax.set_xlabel('a')
ax.set_ylabel('b')

ax.set_aspect('equal', adjustable='box')

plt.show()


/home/francois/sorbetto/sorbetto/ranking/ranking_score.py:436: RuntimeWarning: invalid value encountered in divide
  return satisfying / (satisfying + unsatisfying)


In [8]:
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)  # leave space for sliders

a = np.linspace(0, 1, 101)
b = np.linspace(0, 1, 101)
a2, b2 = np.meshgrid(a, b)


prior_ax = fig.add_axes([0.2, 0.1, 0.6, 0.03])
prior_pos_slider = Slider(
    ax=prior_ax,
    label=r'$\pi_+$',
    valmin=0.0,
    valmax=1.0,
    valinit=0.5,
    valstep=0.01,
    orientation='horizontal'
)

proba_ax = fig.add_axes([0.2, 0.05, 0.6, 0.03])
proba_model_slider = Slider(
    ax=proba_ax,
    label='P(+)',
    valmin=0.0,
    valmax=1.0,
    valinit=0.5,
    valstep=0.01,
    orientation='horizontal'
)

def plot_tile(_=None):
    prior_val = prior_pos_slider.val
    proba_model_val = proba_model_slider.val

    id_entity = encode_id(prior_val, proba_model_val)
    id_entity = id_dict[id_entity]
    flavor = RankingFlavor(entity = entities_list[id_entity],
                       entity_list=entities_list,
                       name = "Rank Flavor - No Skill")

    tile = RankingTile(name = "Rank Tile - No Skill", 
                    parameterization=param,
                    flavor=flavor)
    mat = tile(a2, b2)

    ax.clear()
    ax.imshow(mat, extent=[0, 1, 0, 1], origin='lower', aspect='equal')
    fig.canvas.draw_idle()

prior_pos_slider.on_changed(plot_tile)
proba_model_slider.on_changed(plot_tile)

# initial draw
plot_tile()
plt.show()

# TODO fix dictionary error


KeyError: '0.50_0.50'